### And welcome to Week 4, Day 3 - more LangGraph..

In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain_openai import ChatOpenAI
from typing import TypedDict


In [ ]:
# Our favorite first step! Crew was doing this for us, by the way.
load_dotenv(override=True)


In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()
serper.run("Is Stevia good for you")

In [ ]:
from langchain.agents import Tool

tool_search = Tool(
    name="ingredient_researcher",
    func=serper.run,
    description="Search for food labels, ingredient lists, and nutritional additives to verify health safety."
)

In [ ]:
tool_search.invoke("Is stevia safe?")

In [ ]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def ingredient_alert(text: str):
    """Send a critical health alert regarding food ingredients"""
    alert_message = f"GUTCHECK ALERT: {text}"
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": alert_message})

In [ ]:
tool_push = Tool(
        name="send_health_alert",
        func=ingredient_alert,
        description="Use this tool ONLY when you find a dangerous or banned ingredient that requires an immediate user alert."
    )

tool_push.invoke("High Fructose Corn Syrup found in the ingredient list")

In [ ]:
tools = [tool_search, tool_push]

In [ ]:
# Step 1: Define the State object
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

In [ ]:
# This is different:

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [ ]:

def nutritionist_node(state: State):
    system_message = {
        "role": "system", 
        "content": "You are 'GutCheck'. Use your tools to research ingredients. If you find something dangerous, use the 'send_health_alert' tool immediately."
    }
    messages = [system_message] + state["messages"]
    return {"messages": [llm_with_tools.invoke(messages)]}

graph_builder.add_node("nutritionist", nutritionist_node)
graph_builder.add_node("tools", ToolNode(tools=tools))

In [ ]:
# Step 4: Create Edges


graph_builder.add_conditional_edges( "nutritionist", tools_condition, "tools")

# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "nutritionist")
graph_builder.add_edge(START, "nutritionist")

In [ ]:
# Step 5: Compile the Graph
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]})
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
# Steps 1 and 2
graph_builder = StateGraph(State)


# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def nutritionist_node(state: State):
    print(state)
    system_msg = {"role": "system", "content": "You are 'GutCheck'. Use tools to research ingredients and alert users to dangers."}
    inputs = [system_msg] + state["messages"]
    return {"messages": [llm_with_tools.invoke(inputs)]}

graph_builder.add_node("nutritionist", nutritionist_node)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "nutritionist", tools_condition, "tools")
graph_builder.add_edge("tools", "nutritionist")
graph_builder.add_edge(START, "nutritionist")

# Step 5
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

In [ ]:
graph.get_state(config)

In [ ]:
# Most recent first

list(graph.get_state_history(config))

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

In [ ]:
# Steps 1 and 2
graph_builder = StateGraph(State)


# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def nutritionist_node(state: State):
    print(state)
    system_msg = {"role": "system", "content": "You are 'GutCheck'. Use tools to research ingredients and alert users to dangers."}
    inputs = [system_msg] + state["messages"]
    return {"messages": [llm_with_tools.invoke(inputs)]}

graph_builder.add_node("nutritionist", nutritionist_node)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "nutritionist", tools_condition, "tools")
graph_builder.add_edge("tools", "nutritionist")
graph_builder.add_edge(START, "nutritionist")

# Step 5
graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))
 

In [ ]:
config = {"configurable": {"thread_id": "4"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()